In [18]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [19]:
iris = load_iris()
x = iris.data
y = iris.target

In [20]:
# 붓꽃의 꽃받침과 꽃잎의 [길이, 너비]에 대한 값 -> 붓꽃의 종류 setosa / versicolor / virginica
print(x.shape, y.shape)

(150, 4) (150,)


In [21]:
print(x[0], y[0])

[5.1 3.5 1.4 0.2] 0


In [22]:
scaler = StandardScaler()
x = scaler.fit_transform(x)

In [23]:
print(x[0])

[-0.90068117  1.01900435 -1.34022653 -1.3154443 ]


In [24]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2)

In [25]:
print(x_train.shape, x_test.shape, y_train.shape, y_test.shape)

(120, 4) (30, 4) (120,) (30,)


In [26]:
x_train = x_train.reshape(-1, 30, 4)
y_train = y_train.reshape(-1, 30)

In [27]:
print(x_train.shape, x_test.shape, y_train.shape, y_test.shape)

(4, 30, 4) (30, 4) (4, 30) (30,)


In [28]:
x_train = torch.FloatTensor(x_train)
y_train = torch.LongTensor(y_train)
x_test = torch.FloatTensor(x_test)
y_test = torch.LongTensor(y_test)

In [29]:
class FCN(nn.Module):

  def __init__(self, input_dim, num_classes):
    super(FCN, self).__init__()
    self.fc = nn.Linear(input_dim, 4)
    self.fc2 = nn.Linear(4, num_classes)

  def forward(self, x):
    x = torch.sigmoid(self.fc(x))
    return torch.sigmoid(self.fc2(x))

In [30]:
model = FCN(input_dim=x_train.shape[2], num_classes=3)
print(model)

FCN(
  (fc): Linear(in_features=4, out_features=4, bias=True)
  (fc2): Linear(in_features=4, out_features=3, bias=True)
)


In [31]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [32]:
epochs = 10000
for e in range(epochs):
  for b in range(len(x_train)):
    outputs = model(x_train[b])
    loss = criterion(outputs, y_train[b])
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

  if (e+1) % 1000 == 0:
    print(f'Epoch {e + 1} / {epochs}, Loss : {loss.item()}')

Epoch 1000 / 10000, Loss : 0.6390693783760071
Epoch 2000 / 10000, Loss : 0.567866325378418
Epoch 3000 / 10000, Loss : 0.5559428930282593
Epoch 4000 / 10000, Loss : 0.5527656674385071
Epoch 5000 / 10000, Loss : 0.5518144369125366
Epoch 6000 / 10000, Loss : 0.5515533089637756
Epoch 7000 / 10000, Loss : 0.5514791011810303
Epoch 8000 / 10000, Loss : 0.551456093788147
Epoch 9000 / 10000, Loss : 0.55144864320755
Epoch 10000 / 10000, Loss : 0.5514460206031799


In [33]:
# 테스트
with torch.no_grad():
    # accuracy = 0
    # total = 0
    model.eval()
    # for b in range(len(x_test)):
    #   outputs = model(x_test[b])
    #   _, predicted = torch.max(outputs, 1)
    #   total += y_test[b].size(0)
    #   accuracy += (predicted == y_test[b]).sum().item()
    # print(f'Test Accuracy: {accuracy * 100 / total}')
    correct = 0
    total = 0
    outputs = model(x_test)
    _, predicted = torch.max(outputs.data, 1)
    total += y_test.size(0)
    correct += (predicted == y_test).sum().item()
    print('Accuracy of the model on test data: {} %'.format(100 * correct / total))


Accuracy of the model on test data: 86.66666666666667 %
